# Project 7 

In this project, you will implement the the clustering techniques that you've learned this week. 

#### Step 1: Load the python libraries that you will need for this project 

In [1]:
import pandas as pd 
import matplotlib as plt
import numpy as np
import sklearn
import psycopg2 as psy
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL


#### Step 2: Examine your data 

In [2]:
# Typically, a relational database is organized into tables. Each table should correspond to one entity or concept. 
# Each table is similar to a single CSV file or Pandas dataframe.

In [3]:
PATH = '/Users/cguy/Downloads'
df = pd.read_csv(PATH + '/airports.csv')
df2 = pd.read_csv(PATH + '/Airport_operations.csv')
df3 = pd.read_csv(PATH + '/airport_cancellations.csv')

In [4]:
print df.columns
print df2.columns
print df3.columns


Index([u'Key', u'LocID', u'AP_NAME', u'ALIAS', u'Facility Type', u'FAA REGION',
       u'COUNTY', u'CITY', u'STATE', u'AP Type', u'Latitude', u'Longitude',
       u'Boundary Data Available'],
      dtype='object')
Index([u'airport', u'year', u'departures for metric computation',
       u'arrivals for metric computation', u'percent on-time gate departures',
       u'percent on-time airport departures', u'percent on-time gate arrivals',
       u'average_gate_departure_delay', u'average_taxi_out_time',
       u'average taxi out delay', u'average airport departure delay',
       u'average airborne delay', u'average taxi in delay',
       u'average block delay', u'average gate arrival delay'],
      dtype='object')
Index([u'Airport', u'Year', u'Departure Cancellations',
       u'Arrival Cancellations', u'Departure Diversions',
       u'Arrival Diversions'],
      dtype='object')


In [5]:
#postgres doesn't like capitals or spaces

df.columns = [c.lower().replace(' ','') for c in df.columns] 
df2.columns = [c.lower().replace(' ','') for c in df2.columns]
df3.columns = [c.lower().replace(' ','') for c in df3.columns]


In [6]:
print df.columns
print df2.columns
print df3.columns

Index([u'key', u'locid', u'ap_name', u'alias', u'facilitytype', u'faaregion',
       u'county', u'city', u'state', u'aptype', u'latitude', u'longitude',
       u'boundarydataavailable'],
      dtype='object')
Index([u'airport', u'year', u'departuresformetriccomputation',
       u'arrivalsformetriccomputation', u'percenton-timegatedepartures',
       u'percenton-timeairportdepartures', u'percenton-timegatearrivals',
       u'average_gate_departure_delay', u'average_taxi_out_time',
       u'averagetaxioutdelay', u'averageairportdeparturedelay',
       u'averageairbornedelay', u'averagetaxiindelay', u'averageblockdelay',
       u'averagegatearrivaldelay'],
      dtype='object')
Index([u'airport', u'year', u'departurecancellations', u'arrivalcancellations',
       u'departurediversions', u'arrivaldiversions'],
      dtype='object')


In [7]:
df.head()

,key,locid,ap_name,alias,facilitytype,faaregion,county,city,state,aptype,latitude,longitude,boundarydataavailable
0,3443.0,STX,HENRY E ROHLSEN,Henry E Rohlsen Int'l Airport,Airport,ASO,-VIRGIN ISLANDS-,CHRISTIANSTED,VI,Public Use,17.701556,-64.801722,Yes
1,5088.0,X64,PATILLAS,NaN,Airport,ASO,#NAME?,PATILLAS,PR,Public Use,17.982189,-66.019330,No
2,2886.0,PSE,MERCEDITA,Aeropuerto Mercedita,Airport,ASO,#NAME?,PONCE,PR,Public Use,18.008306,-66.563028,Yes
3,2879.0,VQS,ANTONIO RIVERA RODRIGUEZ,Aeropuerto Antonio Rivera Rodr�guez,Airport,ASO,#NAME?,ISLA DE VIEQUES,PR,Public Use,18.134811,-65.493617,Yes
4,2883.0,X63,HUMACAO,Aeropuerto Regional De Humacao,Airport,ASO,#NAME?,HUMACAO,PR,Public Use,18.138017,-65.800718,Yes


In [8]:
df2.head()

,airport,year,departuresformetriccomputation,arrivalsformetriccomputation,percenton-timegatedepartures,percenton-timeairportdepartures,percenton-timegatearrivals,average_gate_departure_delay,average_taxi_out_time,averagetaxioutdelay,averageairportdeparturedelay,averageairbornedelay,averagetaxiindelay,averageblockdelay,averagegatearrivaldelay
0,ABQ,2004,53971,53818,0.8030,0.7809,0.7921,10.38,9.89,2.43,12.10,2.46,0.83,2.55,10.87
1,ABQ,2005,51829,51877,0.8140,0.7922,0.8001,9.60,9.79,2.29,11.20,2.26,0.89,2.34,10.24
2,ABQ,2006,49682,51199,0.7983,0.7756,0.7746,10.84,9.89,2.16,12.33,2.12,0.84,2.66,11.82
3,ABQ,2007,53255,53611,0.8005,0.7704,0.7647,11.29,10.34,2.40,12.95,2.19,1.29,3.06,12.71
4,ABQ,2008,49589,49512,0.8103,0.7844,0.7875,10.79,10.41,2.41,12.32,1.82,1.03,2.79,11.48


In [9]:
df3.head()

,airport,year,departurecancellations,arrivalcancellations,departurediversions,arrivaldiversions
0,ABQ,2004.0,242.0,235.0,71.0,46.0
1,ABQ,2005.0,221.0,190.0,61.0,33.0
2,ABQ,2006.0,392.0,329.0,71.0,124.0
3,ABQ,2007.0,366.0,304.0,107.0,45.0
4,ABQ,2008.0,333.0,300.0,79.0,42.0


### Intro: Write a problem statement / aim for this project

Aims:

- analyze the operations of major airports around the country
- help FAA to cut down on delays nationwide
- Understand the characteristics and groupings of airports based on a dataset of departure and operational delays
- understand the distribution, characteristics, and components of individual airports operations that are leading to these delays
- Organize and store their data so that they can easily understand it after your consulting work is done
- modeling using principal component analysis

Problem Statment:

Which airports and operational characteristics should the FAA target to decrease delays?





Use your skills with PCA to uncover the core components of operations related to delays.

recommendations as to which airports and operational characteristics they should target to decrease delays.

Which operational features are most correlated with delays?

What should the airport's next steps be?

What operational factors are most directly correlated to delays?

Take a look at airports groupings - are there any relationships by region? Size?

Conduct a PCA to discover the principal components behind departure delays

### Part 1: Create a PostgreSQL database 

#### 1. Let's create a database where we can house our airport data

Postgres is a server-based database, which means that
there is a server process running (bind to some port), that
accepts requests (i.e. SQL queries). Your process connects
to the server and sends queries to the server which itself is
responsible to update the database files.

The sql extension is already loaded. To reload it, use:
  %reload_ext sql
Connect to the database. Note - enter your own connection string. For help on how to load the raw CSV file into a Postgres database, please refer to the documenatation in the lesson plan on previous SQL lessons.
Create your database in postrgres:
psql
user=# create database wine
user=# \quit

In [11]:
engine = create_engine('postgresql://localhost:5432')

In [12]:
# %load_ext sql
%reload_ext sql

/Users/cguy/anaconda/lib/python2.7/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/Users/cguy/anaconda/lib/python2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


Load our csv files into tables

In [ ]:
#ALREADY DONE ABOVE

In [13]:
df.to_sql('airport_info', engine)
df2.to_sql('airport_ops', engine)
df3.to_sql('airport_cancels', engine)

In [14]:
%%sql postgresql://localhost:5432/
        
SELECT * FROM airport_info LIMIT 5

5 rows affected.


index,key,locid,ap_name,alias,facilitytype,faaregion,county,city,state,aptype,latitude,longitude,boundarydataavailable
0,3443.0,STX,HENRY E ROHLSEN,Henry E Rohlsen Int'l Airport,Airport,ASO,-VIRGIN ISLANDS-,CHRISTIANSTED,VI,Public Use,17.701556,-64.801722,Yes
1,5088.0,X64,PATILLAS,None,Airport,ASO,#NAME?,PATILLAS,PR,Public Use,17.982189,-66.01933,No
2,2886.0,PSE,MERCEDITA,Aeropuerto Mercedita,Airport,ASO,#NAME?,PONCE,PR,Public Use,18.008306,-66.563028,Yes
3,2879.0,VQS,ANTONIO RIVERA RODRIGUEZ,Aeropuerto Antonio Rivera Rodr�guez,Airport,ASO,#NAME?,ISLA DE VIEQUES,PR,Public Use,18.134811,-65.493617,Yes
4,2883.0,X63,HUMACAO,Aeropuerto Regional De Humacao,Airport,ASO,#NAME?,HUMACAO,PR,Public Use,18.138017,-65.800718,Yes


In [15]:
%%sql 

SELECT * FROM airport_ops LIMIT 5

5 rows affected.


index,airport,year,departuresformetriccomputation,arrivalsformetriccomputation,percenton-timegatedepartures,percenton-timeairportdepartures,percenton-timegatearrivals,average_gate_departure_delay,average_taxi_out_time,averagetaxioutdelay,averageairportdeparturedelay,averageairbornedelay,averagetaxiindelay,averageblockdelay,averagegatearrivaldelay
0,ABQ,2004,53971,53818,0.803,0.7809,0.7921,10.38,9.89,2.43,12.1,2.46,0.83,2.55,10.87
1,ABQ,2005,51829,51877,0.814,0.7922,0.8001,9.6,9.79,2.29,11.2,2.26,0.89,2.34,10.24
2,ABQ,2006,49682,51199,0.7983,0.7756,0.7746,10.84,9.89,2.16,12.33,2.12,0.84,2.66,11.82
3,ABQ,2007,53255,53611,0.8005,0.7704,0.7647,11.29,10.34,2.4,12.95,2.19,1.29,3.06,12.71
4,ABQ,2008,49589,49512,0.8103,0.7844,0.7875,10.79,10.41,2.41,12.32,1.82,1.03,2.79,11.48


In [23]:
%%sql 

SELECT * FROM airport_cancels LIMIT 5

5 rows affected.


index,airport,year,departurecancellations,arrivalcancellations,departurediversions,arrivaldiversions
0,ABQ,2004.0,242.0,235.0,71.0,46.0
1,ABQ,2005.0,221.0,190.0,61.0,33.0
2,ABQ,2006.0,392.0,329.0,71.0,124.0
3,ABQ,2007.0,366.0,304.0,107.0,45.0
4,ABQ,2008.0,333.0,300.0,79.0,42.0


In [25]:
# Obsolete (returns all rows in References that do not have rows in CurrentFIMS):

In [24]:
%%sql 

SELECT ao.index, ao.airport FROM airport_ops ao
LEFT JOIN airport_cancels ac ON ao.index = ac.index AND ao.airport = ac.airport
WHERE ac.index IS NULL AND ac.airport IS NULL;


599 rows affected.


index,airport
242,GYY
243,GYY
244,GYY
245,GYY
246,GYY
247,GYY
248,GYY
249,GYY
250,GYY
251,GYY


In [26]:
# Don't exist yet (returns all rows in CurrentFIMS that do not have rows in References):

In [27]:
%%sql 

SELECT ac.index, ac.airport FROM airport_cancels ac
LEFT JOIN airport_ops ao ON ao.index = ac.index AND ao.airport = ac.airport
WHERE ao.index IS NULL AND ao.airport IS NULL;

563 rows affected.


index,airport
242,HNL
243,HNL
244,HNL
245,HNL
246,HNL
247,HNL
248,HNL
249,HNL
250,HNL
251,HNL


In [ ]:
#two tables don't match up.....will need to address

Query the database for our intial data

In [ ]:
%%sql 



In [ ]:
%%sql



In [ ]:
%%sql



In [ ]:
%%sql 



In [ ]:
%%sql



In [ ]:
%%sql 



#### 1.2 What are the risks and assumptions of our data? 

-A certain degree of delay is expected in airport operations, however the FAA is noticing significant delays with certain airports
-When a flight takes off, it's departure delay is recorded in minutes, as well as operational data relating to this delay

-At the end of the year, this data is averaged out for each airport. Your datasets have these averaged for a 10 year range between 2004 and 2014
-Over this 10 year range, some delay times have not improved or have worsened.

-In this project, we're going to be using three different datasets related to airport operations. These include:
    -a dataset detailing the arrival and departure delays/diversions by airport, 
    -a dataset that provides metrics related to arrivals and departures for each airport, 
    -and a dataset that details names and characteristics for each airport code.

### Part 2: Exploratory Data Analysis

#### 2.1 Plot and Describe the Data

#### Are there any unique values? 

### Part 3: Data Mining

#### 3.1 Create Dummy Variables

#### 3.2 Format and Clean the Data

### Part 4: Define the Data

#### 4.1 Confirm that the dataset has a normal distribution. How can you tell? 

#### 4.2 Find correlations in the data

#### 4.3 What is the value of understanding correlations before PCA? 

Answer: 

#### 4.4 Validate your findings using statistical analysis

#### 4.5 How can you improve your overall analysis? 

Answer: 

### Part 5: Perform a PCA

#### 5.1 Conduct the PCA

### Part 6: Additional Analysis
Include any other models you'd like to run here. These can include regressions, classifications, or clusters. 

### Part 7: Write an analysis plan of your findings 

Create a writeup on the interpretation of findings including an executive summary with conclusions and next steps. Put it on your blog, and include the link here.

Which operational features are most correlated with delays?

What should the airport's next steps be?

### Bonus: Copy your Database to AWS 

Make sure to properly document all of the features of your dataset

### Bonus: Create a 3-Dimensional Plot of your new dataset with PCA applied